In [1]:
from pathlib import Path
import os
import kaggle
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
import optuna

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

VERSION = 5
OUTPUT = f'extra_tree_clf_submisson_v{VERSION}.csv'

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

seed = 42
splits = 5
np.random.seed(seed)

In [5]:
partial_pipeline = make_pipeline(ColumnTransformer([('drop', 'drop',
                                  ['iv(g)', 't', 'b', 'n',
                                   'lOCode', 'v', 'branchCount',
                                   'e', 'i', 'lOComment'])],
                                remainder='passthrough'), SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, 100),
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'random_state': trial.suggest_categorical('random_state', [seed]),
        'max_depth': trial.suggest_int('max_depth', 4, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        'criterion': trial.suggest_categorical("criterion", ["gini"]),
        'max_features': trial.suggest_categorical('max_features', ["sqrt", "log2"]),
    }

    pipeline = make_pipeline(partial_pipeline, ExtraTreesClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=splits, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2023-10-14 17:50:49,053] A new study created in memory with name: no-name-4fdde989-0b97-47ad-9590-103c91aa327f
Best trial: 0. Best value: 0.780492:   3%|█████▍                                                                                                                                                              | 1/30 [00:30<14:43, 30.45s/it]

[I 2023-10-14 17:51:19,505] Trial 0 finished with value: 0.7804917426976707 and parameters: {'n_estimators': 500, 'n_jobs': -1, 'random_state': 42, 'max_depth': 26, 'min_samples_split': 144, 'min_samples_leaf': 14, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7804917426976707.


Best trial: 0. Best value: 0.780492:   7%|██████████▉                                                                                                                                                         | 2/30 [00:42<09:11, 19.69s/it]

[I 2023-10-14 17:51:31,660] Trial 1 finished with value: 0.7796452845399605 and parameters: {'n_estimators': 200, 'n_jobs': -1, 'random_state': 42, 'max_depth': 38, 'min_samples_split': 105, 'min_samples_leaf': 32, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7804917426976707.


Best trial: 0. Best value: 0.780492:  10%|████████████████▍                                                                                                                                                   | 3/30 [01:02<08:58, 19.95s/it]

...

[I 2023-10-14 18:05:30,223] Trial 27 finished with value: 0.7807419588392888 and parameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 42, 'max_depth': 45, 'min_samples_split': 114, 'min_samples_leaf': 13, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 26 with value: 0.7839128458038738.


Best trial: 26. Best value: 0.783913:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [15:17<00:46, 46.25s/it]

[I 2023-10-14 18:06:06,913] Trial 28 finished with value: 0.7810192553564007 and parameters: {'n_estimators': 300, 'n_jobs': -1, 'random_state': 42, 'max_depth': 40, 'min_samples_split': 135, 'min_samples_leaf': 11, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 26 with value: 0.7839128458038738.


Best trial: 26. Best value: 0.783913: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [16:17<00:00, 32.59s/it]

[I 2023-10-14 18:07:06,677] Trial 29 finished with value: 0.7802684629387135 and parameters: {'n_estimators': 500, 'n_jobs': -1, 'random_state': 42, 'max_depth': 46, 'min_samples_split': 72, 'min_samples_leaf': 18, 'criterion': 'gini', 'max_features': 'log2'}. Best is trial 26 with value: 0.7839128458038738.


In [7]:
study.best_params, study.best_value

({'n_estimators': 300,
  'n_jobs': -1,
  'random_state': 42,
  'max_depth': 45,
  'min_samples_split': 112,
  'min_samples_leaf': 1,
  'criterion': 'gini',
  'max_features': 'log2'},
 0.7839128458038738)

In [8]:
pipeline = make_pipeline(partial_pipeline, ExtraTreesClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['iv(g)',
                                                                    't', 'b',
                                                                    'n',
                                                                    'lOCode',
                                                                    'v',
                                                                    'branchCount',
                                                                    'e', 'i',
                                                                    'lOComment'])])),
                                 ('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('extratreesclassifier',
                 ExtraTreesClassifier(max_depth=45, max_features='log2',
                                      min_samples_split=112, n_estimators=300,
                                      n_jobs=-1, random_state=42))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)